In [3]:
import torch
import clip
from PIL import Image

In [9]:
# TODO: get rid of this dirty hack
import os
os.getcwd()
import sys
sys.path.append("/Users/htkumar/clip")

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:20<00:00, 17.2MiB/s]


In [11]:
type(model)

clip.model.CLIP

In [15]:
image = preprocess(Image.open("../CLIP.png")).unsqueeze(0)

In [16]:
image.shape

torch.Size([1, 3, 224, 224])

In [17]:
text = clip.tokenize(["a diagram", "a dog", "a cat"])

In [18]:
text.shape

torch.Size([3, 77])

In [20]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x16566a0e0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [22]:

image_features = model.encode_image(image)
print(image_features.shape)
text_features = model.encode_text(text)
print(text_features.shape)

torch.Size([1, 512])
torch.Size([3, 512])


In [32]:
a = (100. * (image_features @ text_features.T)).softmax(dim=-1)
a

tensor([[1., 0., 0.]], grad_fn=<SoftmaxBackward0>)

In [30]:
model(image, text)

(tensor([[25.5528, 20.0899, 19.7495]], grad_fn=<MmBackward0>),
 tensor([[25.5528],
         [20.0899],
         [19.7495]], grad_fn=<TBackward0>))

In [33]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [36]:
clip.tokenize(["A deer is here", "A beer is here"]).shape

torch.Size([2, 77])

### Zero shot prediction

In [37]:
from torchvision.datasets import CIFAR100

In [38]:
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

100%|██████████| 169001437/169001437 [00:02<00:00, 60161875.71it/s]


Extracting /Users/htkumar/.cache/cifar-100-python.tar.gz to /Users/htkumar/.cache


In [39]:
cifar100[0]

(<PIL.Image.Image image mode=RGB size=32x32>, 49)

In [40]:
len(cifar100)

10000

In [41]:
#Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes])

In [42]:
image_input.shape, text_inputs.shape

(torch.Size([1, 3, 224, 224]), torch.Size([100, 77]))

In [43]:
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

In [44]:
image_features.shape, text_features.shape

(torch.Size([1, 512]), torch.Size([100, 512]))

In [49]:
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100. * image_features @ text_features.T).softmax(dim=-1)

In [50]:
similarity

tensor([[1.1976e-03, 5.4627e-03, 4.8136e-03, 1.8082e-04, 1.8419e-04, 3.4513e-04,
         5.7727e-04, 8.2103e-03, 9.9322e-05, 3.7705e-04, 1.0402e-03, 8.2095e-04,
         3.4284e-05, 1.0731e-04, 1.9522e-03, 1.4169e-04, 1.7230e-03, 4.6303e-05,
         4.2151e-03, 9.5372e-04, 2.7544e-04, 1.2663e-03, 3.1629e-04, 5.2210e-05,
         6.0814e-04, 1.6915e-04, 7.3051e-03, 1.7485e-02, 5.9205e-04, 4.7084e-03,
         1.2483e-05, 4.3250e-04, 8.4593e-03, 1.0473e-04, 1.4937e-05, 5.7394e-04,
         9.1998e-04, 1.1684e-04, 1.0331e-04, 2.5124e-04, 1.1973e-03, 6.8170e-03,
         1.7164e-02, 8.7136e-04, 1.8750e-02, 1.4284e-04, 3.7086e-04, 1.1403e-04,
         1.0234e-04, 4.6626e-04, 6.6153e-03, 2.9241e-03, 6.3651e-04, 7.6152e-04,
         1.2286e-04, 1.4347e-04, 8.5427e-04, 1.4071e-04, 2.5623e-04, 1.5453e-03,
         8.2367e-04, 1.7338e-03, 5.6944e-04, 3.3849e-04, 4.9928e-04, 6.9673e-04,
         4.8177e-04, 6.3734e-04, 7.4546e-06, 1.2440e-05, 1.7304e-04, 7.2544e-05,
         1.1135e-03, 5.2110e

In [51]:
similarity.shape

torch.Size([1, 100])

In [52]:
similarity[0]

tensor([1.1976e-03, 5.4627e-03, 4.8136e-03, 1.8082e-04, 1.8419e-04, 3.4513e-04,
        5.7727e-04, 8.2103e-03, 9.9322e-05, 3.7705e-04, 1.0402e-03, 8.2095e-04,
        3.4284e-05, 1.0731e-04, 1.9522e-03, 1.4169e-04, 1.7230e-03, 4.6303e-05,
        4.2151e-03, 9.5372e-04, 2.7544e-04, 1.2663e-03, 3.1629e-04, 5.2210e-05,
        6.0814e-04, 1.6915e-04, 7.3051e-03, 1.7485e-02, 5.9205e-04, 4.7084e-03,
        1.2483e-05, 4.3250e-04, 8.4593e-03, 1.0473e-04, 1.4937e-05, 5.7394e-04,
        9.1998e-04, 1.1684e-04, 1.0331e-04, 2.5124e-04, 1.1973e-03, 6.8170e-03,
        1.7164e-02, 8.7136e-04, 1.8750e-02, 1.4284e-04, 3.7086e-04, 1.1403e-04,
        1.0234e-04, 4.6626e-04, 6.6153e-03, 2.9241e-03, 6.3651e-04, 7.6152e-04,
        1.2286e-04, 1.4347e-04, 8.5427e-04, 1.4071e-04, 2.5623e-04, 1.5453e-03,
        8.2367e-04, 1.7338e-03, 5.6944e-04, 3.3849e-04, 4.9928e-04, 6.9673e-04,
        4.8177e-04, 6.3734e-04, 7.4546e-06, 1.2440e-05, 1.7304e-04, 7.2544e-05,
        1.1135e-03, 5.2110e-05, 1.3243e-

In [55]:
# print the result
similarity[0].topk??

Docstring:
topk(k, dim=None, largest=True, sorted=True) -> (Tensor, LongTensor)

See :func:`torch.topk`
Type:      builtin_function_or_method

In [56]:
values, indices = similarity[0].topk(5)

In [63]:
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100.*value.item():.2f}%")

           snake: 65.31%
          turtle: 12.29%
    sweet_pepper: 3.83%
          lizard: 1.87%
       crocodile: 1.75%


In [64]:
from tqdm import tqdm

In [65]:
root = os.path.expanduser("~/.cache")
train = CIFAR100(root, download=True, train=True, transform=preprocess)
test = CIFAR100(root, download=True, train=False, transform=preprocess)

Files already downloaded and verified
Files already downloaded and verified


In [67]:
# tqdm??

In [68]:
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from tqdm import tqdm

In [69]:
def get_features(dataset):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images)
            all_features.append(features)
            all_labels.append(labels)
    
    return torch.cat(all_features).numpy(), torch.cat(all_labels).numpy()

In [70]:
train_features, train_labels = get_features(train)

100%|██████████| 500/500 [10:01<00:00,  1.20s/it]


In [71]:
test_features, test_labels = get_features(test)

100%|██████████| 100/100 [02:06<00:00,  1.26s/it]
